In [33]:
import os
import subprocess
import pandas as pd
import platform
import pathlib
import numpy as np
from annotations import *
from extract_video_features import *
from extract_audio_features import *
import cv2
import seaborn as sns
from scipy.stats import kurtosis, skew
from sklearn.metrics import f1_score
import importlib
from tqdm import tqdm
import ordinal_classification as o_c
from sklearn.metrics import confusion_matrix


In [34]:
#get current directory
cwd = os.getcwd()

In [35]:
cwd

'/Users/valentinadiproietto/filrouge'

In [36]:
OpenFace_folder = '/Users/valentinadiproietto/OpenFace'
filename_annotations = 'https://docs.google.com/spreadsheets/d/1Rqu1sJiD-ogc4a6R491JTiaYacptOTqh6DKqhwTa8NA/gviz/tq?tqx=out:csv&sheet=Template'

In [37]:
Video_folder = '/Users/valentinadiproietto/Desktop/video_stress'

In [38]:
video_paths, video_names = get_videos(Video_folder)

In [39]:
video_names

['WIN_20210331_21_22_52_Pro',
 'WIN_20210329_14_13_45_Pro',
 'WIN_20210406_18_49_10_Pro',
 'WIN_20210408_14_11_32_Pro',
 'WIN_20210408_15_20_51_Pro',
 'WIN_20210404_10_58_27_Pro',
 'WIN_20210414_06_24_52_Pro',
 'WIN_20210406_15_06_15_Pro',
 'WIN_20210417_14_53_12_Pro',
 'WIN_20210413_15_38_01_Pro',
 'WIN_20210408_11_48_58_Pro',
 'WIN_20210408_16_04_32_Pro',
 'WIN_20210329_10_16_02_Pro',
 'WIN_20210323_19_17_40_Pro',
 'WIN_20210409_10_26_11_Pro',
 'Test_pour_AFPA',
 'WIN_20210405_15_09_16_Pro',
 'WIN_20210407_14_54_56_Pro_edit2',
 'WIN_20210406_21_05_52_Pro',
 'WIN_20210403_18_49_15_Pro',
 'WIN_20210408_14_02_19_Pro',
 'WIN_20210415_15_41_24_Pro',
 'WIN_20210406_18_35_52_Pro',
 'WIN_20210402_14_27_50_Pro',
 'WIN_20210407_09_04_05_Pro',
 'WIN_20210402_19_04_53_Pro',
 'WIN_20210416_08_06_54_Pro',
 'Video_1',
 'WIN_20210408_14_00_44_Pro',
 'WIN_20210404_21_41_12_Pro',
 'WIN_20210330_13_10_29_Pro']

In [44]:
list_dataframes = []
for i in video_names: 
    list_dataframes.append(create_dataframe_video('/Users/valentinadiproietto/OpenFace/processed/', i))


## PRELIMINARY STUDY OF FEATURES

In [45]:
for df in list_dataframes:
    print(df[df["confidence"]<0.7]["success"].value_counts())

0    226
Name: success, dtype: int64
0    339
Name: success, dtype: int64
Series([], Name: success, dtype: int64)
0    12
Name: success, dtype: int64
0    118
Name: success, dtype: int64
0    11
Name: success, dtype: int64
0    340
Name: success, dtype: int64
0    35
Name: success, dtype: int64
Series([], Name: success, dtype: int64)
0    121
Name: success, dtype: int64
0    506
Name: success, dtype: int64
0    15
Name: success, dtype: int64
0    1
Name: success, dtype: int64
0    9
Name: success, dtype: int64
0    264
Name: success, dtype: int64
Series([], Name: success, dtype: int64)
0    22
Name: success, dtype: int64
Series([], Name: success, dtype: int64)
0    208
Name: success, dtype: int64
0    122
Name: success, dtype: int64
0    5
Name: success, dtype: int64
Series([], Name: success, dtype: int64)
0    70
Name: success, dtype: int64
0    14279
1       24
Name: success, dtype: int64
Series([], Name: success, dtype: int64)
0    12
Name: success, dtype: int64
Series([], Name: suc

In every dataframe if confidence is less then 0.7, success is always 0. If confidence is between 0.7 and 0.8, there are cases of success. Therefore I think that one can avoid using the confidence column, we can consider that sucess is a goo way to distinguish between cases of success or not. Be carefull, even if success is equal to zero there are many values that are not equal to zero. 

In [46]:
len(list_dataframes)

31

In [47]:
for i in list_dataframes:
    print(check_success(i))

98.34157695664824
93.96474986647677
100.0
99.9474444882407
99.22779922779922
99.96171516079633
95.07823613086771
99.87678987678987
100.0
99.51358405504804
98.44551626678135
99.9315399484536
99.9953288490284
99.96298428888706
98.88920191572423
100.0
99.88188260948573
100.0
98.7196683331301
99.5310891693035
99.97950399672064
100.0
99.69153483453047
47.2531935317138
100.0
99.9057048561999
100.0
100.0
100.0
99.08576533650982
99.39536601126188


The video "'WIN_20210402_14_27_50_Pro'" is problematic for OpenFace, in less than 50% of the frames OpenFace succeded in detecting a face, but in all the other cases OpenFace was succesful, what do we do with this video? We erase the video.

In [48]:
df = list_dataframes[0]

The columns 'AUXX_c' for XX in  ['01','02', '04', '05', '06', '07', '09', '10','12', '14', '15', '17', '20', '23', '25','26', '28', '45'] contain only 0's and 1's where 1 correspond to the presence of the corresponding AUXX, while 0 represents the absence. The columns 'AUXX_r' for XX in  for XX in  ['01','02', '04', '05', '06', '07', '09', '10','12', '14', '15', '17', '20', '23', '25','26', '45'] represent the intensity from 1 to 5 of the respective AU (with continuous value in between). REMARK: OpenFace does not compute the intesity for AU2 which is the lip suck.

In [50]:
count =0
for i in df.columns:
    if i.startswith("AU"):
        count+=1
print(f"There are {count} columns corresponding to AU, {count-count//2} which mesure the intensity and {count//2} that measure if the AU is detected")               



There are 35 columns corresponding to AU, 18 which mesure the intensity and 17 that measure if the AU is detected


In [49]:
print(df.loc[df['AU01_c']==0].shape)

df.loc[df['AU01_c']==0].loc[df['AU01_r']!=0]['AU01_r']

(9978, 714)


1        0.01
2        0.04
3        0.04
4        0.03
8        0.01
         ... 
13709    0.04
13710    0.04
13721    0.05
13722    0.05
13723    0.05
Name: AU01_r, Length: 3422, dtype: float64

The reason is explained in the github "NOTE that the intensity and presence predictors have been trained separately and on slightly different datasets, this means that the predictions of both might not always be consistent (e.g. the presence model could be predicting AU as not being present, but the intensity model could be predicting its value above 1)."

ABOUT gaze: to extract it uses a Constrained Local Neural Field (CLNF) landmark detector with which it determined the eyes landmarks, and then comput the gaze direction for each eye and the angle. 

gaze_0_x, gaze_0_y, gaze_0_z Eye gaze direction vector in world coordinates for eye 0 (normalized), eye 0 is the leftmost eye in the image (think of it as a ray going from the left eye in the image in the direction of the eye gaze)

gaze_1_x, gaze_1_y, gaze_1_z Eye gaze direction vector in world coordinates for eye 1 (normalized), eye 1 is the rightmost eye in the image (think of it as a ray going from the right eye in the image in the direction of the eye gaze)

gaze_angle_x, gaze_angle_y Eye gaze direction in radians in world coordinates averaged for both eyes and converted into more easy to use format than gaze vectors. If a person is looking left-right this will results in the change of gaze_angle_x (from positive to negative) and, if a person is looking up-down this will result in change of gaze_angle_y (from negative to positive), if a person is looking straight ahead both of the angles will be close to 0 (within measurement error).

eye_lmk_x_0, eye_lmk_x_1,... eye_lmk_x55, eye_lmk_y_1,... eye_lmk_y_55 location of 2D eye region landmarks in pixels. The landmark index can be found below

eye_lmk_X_0, eye_lmk_X_1,... eye_lmk_X55, eye_lmk_Y_0,... eye_lmk_Z_55 location of 3D eye region landmarks in millimeters. The landmark index can be found below


THEREFORE WE CAN TAKE gaze_0_x, gaze_0_y, gaze_0_z, gaze_1_x, gaze_1_y, gaze_1_z, gaze_angle_x, gaze_angle_y

OR EVEN SIMPLER gaze_angle_x, gaze_angle_y, angle averaged over the two eyes

HEAD POSE, this is calculated from the facial landmarks extracted with a  Convolutional Experts Constrained Local Model (CE-CLM) 

pose_Tx, pose_Ty, pose_Tz the location of the head with respect to camera in millimeters (positive Z is away from the camera)

pose_Rx, pose_Ry, pose_Rz Rotation is in radians around X,Y,Z axes with the convention R = Rx \* Ry \* Rz, left-handed positive sign. This can be seen as pitch (Rx), yaw (Ry), and roll (Rz). The rotation is in world coordinates with camera being the origin. x-axis=left-right, z-axis =in-out, y-axis = up-down, the Rx, Rz, Ry represent the angle of rotation with respect to a given axis.

So to see how much the head moves, I think we need both.

The feature that starts with p are parameters used in the  point distribution model (PDM) which is used to detect facial landmark and the position of the face. They are parameters used by the model, so I think we can avoid to use them. The author says "Local parameters (34 non-rigid) are parameters controlling non-rigid face shape, in other words things like facial expression and face shape (long, vs round, vs wide faces etc.). In my thesis they are q in equation 4.6 and are explained in detail in section 4.2.2 (https://www.cl.cam.ac.uk/~tb346/pub/thesis/phd_thesis.pdf)

If you are familiar with Principal Component Analysis (PCA), they are the dimensionality reduced components of the 68 facial landmarks. So individually each can represent a particular basis which roughly corresponds to an expression or face shape, but are not always easily mapped to one. If you want to visualize them and see their effect, have a look at matlab_version/pdm_generation/visualise_PDMs.m, this will visualize the first 5 ps and what effect on the face they have.

The global parameters, on the other hand, control the placement of the face in the image and are responsible for translation (x and y), scale/size, and the rotation of the face."

In [51]:
pd.options.display.max_seq_items = 2000
df.columns

Index(['frame', 'face_id', 'timestamp', 'confidence', 'success', 'gaze_0_x',
       'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z',
       'gaze_angle_x', 'gaze_angle_y', 'eye_lmk_x_0', 'eye_lmk_x_1',
       'eye_lmk_x_2', 'eye_lmk_x_3', 'eye_lmk_x_4', 'eye_lmk_x_5',
       'eye_lmk_x_6', 'eye_lmk_x_7', 'eye_lmk_x_8', 'eye_lmk_x_9',
       'eye_lmk_x_10', 'eye_lmk_x_11', 'eye_lmk_x_12', 'eye_lmk_x_13',
       'eye_lmk_x_14', 'eye_lmk_x_15', 'eye_lmk_x_16', 'eye_lmk_x_17',
       'eye_lmk_x_18', 'eye_lmk_x_19', 'eye_lmk_x_20', 'eye_lmk_x_21',
       'eye_lmk_x_22', 'eye_lmk_x_23', 'eye_lmk_x_24', 'eye_lmk_x_25',
       'eye_lmk_x_26', 'eye_lmk_x_27', 'eye_lmk_x_28', 'eye_lmk_x_29',
       'eye_lmk_x_30', 'eye_lmk_x_31', 'eye_lmk_x_32', 'eye_lmk_x_33',
       'eye_lmk_x_34', 'eye_lmk_x_35', 'eye_lmk_x_36', 'eye_lmk_x_37',
       'eye_lmk_x_38', 'eye_lmk_x_39', 'eye_lmk_x_40', 'eye_lmk_x_41',
       'eye_lmk_x_42', 'eye_lmk_x_43', 'eye_lmk_x_44', 'eye_lmk_x_45',
       'eye_lm

Therefor we have decide to keep only the following features: features_to_keep = [
    'frame','face_id','timestamp','confidence','success',

    'AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r',
    'AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', 'AU28_c', 'AU45_c',

    'gaze_0_x','gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z',
    'gaze_angle_x', 'gaze_angle_y', 
    'pose_Tx', 'pose_Ty', 'pose_Tz','pose_Rx', 'pose_Ry', 'pose_Rz',

    'type_candidat','sexe','video_name','stress_global','stress','diapo'
]

## ELIMINATE PROBLEMATIC VIDEOS FROM THE LIST 

In [52]:
video_names.remove('WIN_20210329_14_13_45_Pro')
video_names.remove('WIN_20210402_14_27_50_Pro')


In [53]:
len(video_names)

29

In [54]:
list_df_max = []
for v_name in tqdm(video_names):
    df_annoted = get_df_video_with_annotations('/Users/valentinadiproietto/OpenFace/processed/', v_name, filename_annotations, "max")
    list_df_max.append(eliminate_features(df_annoted))


    

100%|██████████| 29/29 [03:44<00:00,  7.75s/it]


## 3 JUNE 2021, ADDING FIRST AND SECOND DERIVATIVES


In [56]:
df_with_deriv = []
for i in list_df_max:
    to_drop = ['frame','face_id','timestamp','confidence','success', 'type_candidat']
    i = add_frameTimeWindow(i)
    i = i.drop(to_drop, axis = 1)
    i = add_derivatives_drop_spatial(i)
    df_with_deriv.append(i)


In [57]:
groupby_features= ['video_name','stress_global','stress','frameTimeWindow','sexe']

df_total = pd.concat(df_with_deriv)
df_total = df_total.drop('diapo', axis = 1)


In [58]:

df_total = df_total.groupby(groupby_features).agg(['mean']).reset_index()
df_total.columns= df_total.columns.map('_'.join).str.strip('_')
df_total[['stress']].value_counts()

stress
1.0       2377
0.0       2015
2.0        633
3.0         20
dtype: int64

In [59]:
x = df_total.drop(['video_name','stress_global',	'stress','frameTimeWindow'], axis = 1)
x.columns

Index(['sexe', 'AU01_r_mean', 'AU02_r_mean', 'AU04_r_mean', 'AU05_r_mean',
       'AU06_r_mean', 'AU07_r_mean', 'AU09_r_mean', 'AU10_r_mean',
       'AU12_r_mean', 'AU14_r_mean', 'AU15_r_mean', 'AU17_r_mean',
       'AU20_r_mean', 'AU23_r_mean', 'AU25_r_mean', 'AU26_r_mean',
       'AU45_r_mean', 'AU01_c_mean', 'AU02_c_mean', 'AU04_c_mean',
       'AU05_c_mean', 'AU06_c_mean', 'AU07_c_mean', 'AU09_c_mean',
       'AU10_c_mean', 'AU12_c_mean', 'AU14_c_mean', 'AU15_c_mean',
       'AU17_c_mean', 'AU20_c_mean', 'AU23_c_mean', 'AU25_c_mean',
       'AU26_c_mean', 'AU28_c_mean', 'AU45_c_mean', 'AU01_r_de_mean',
       'AU01_r_de_de_mean', 'AU02_r_de_mean', 'AU02_r_de_de_mean',
       'AU04_r_de_mean', 'AU04_r_de_de_mean', 'AU05_r_de_mean',
       'AU05_r_de_de_mean', 'AU06_r_de_mean', 'AU06_r_de_de_mean',
       'AU07_r_de_mean', 'AU07_r_de_de_mean', 'AU09_r_de_mean',
       'AU09_r_de_de_mean', 'AU10_r_de_mean', 'AU10_r_de_de_mean',
       'AU12_r_de_mean', 'AU12_r_de_de_mean', 'AU14_r_de_

In [60]:
#Replace string values
x.sexe = x.sexe.replace('H',0)
x.sexe = x.sexe.replace('F',1)

y = df_total[['stress']]



from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 42)

y_train = y_train.values.squeeze()
y_test = y_test.values.squeeze()


In [61]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 42)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)


print("F1 score = ", f1_score(y_test, y_pred, average='weighted'))

F1 score =  0.7766900508619785


In [62]:
## SPLIT TRAIN AND TEST W RT VIDEOS

In [63]:
video_names_train = video_names[0:21]

In [65]:
video_names_test =  video_names[21:]

In [66]:
df_total_drop = df_total.drop(['stress_global','frameTimeWindow'], axis = 1)
df_total_drop.sexe = df_total_drop.sexe.replace('H',0)
df_total_drop.sexe = df_total_drop.sexe.replace('F',1)

In [69]:
 df_train = df_total_drop[df_total_drop["video_name"].isin(video_names_train)]

In [70]:
df_test =df_total_drop[df_total_drop["video_name"].isin(video_names_test)]

In [71]:
x_train = df_train.drop(['stress', 'video_name'], axis =1)
y_train = df_train[['stress']]
x_test = df_test.drop(['stress', 'video_name'], axis =1)
y_test = df_test[['stress']]




y_train = y_train.values.squeeze()
y_test = y_test.values.squeeze()

In [72]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 42)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)


print("F1 score = ", f1_score(y_test, y_pred, average='weighted'))

F1 score =  0.37453722297806136


## 8 JUIN 2021, CV WITH LEAVE ONE VIDEO OUT 

In [11]:
list_df_max = []
for v_name in tqdm(video_names):
    df_annoted = get_df_video_with_annotations('/Users/valentinadiproietto/OpenFace/processed/', v_name, filename_annotations, "max")
    list_df_max.append(eliminate_features(df_annoted))

100%|██████████| 29/29 [02:23<00:00,  4.96s/it]


In [71]:
df_with_deriv = []
for i in list_df_max:
    to_drop = ['frame','face_id','timestamp','confidence','success', 'type_candidat']
    i = add_frameTimeWindow(i)
    i = i.drop(to_drop, axis = 1)
    i = add_derivatives_drop_spatial(i)
    df_with_deriv.append(i)

In [72]:
groupby_features= ['video_name','stress_global','stress','frameTimeWindow','sexe']

df_total = pd.concat(df_with_deriv)
df_total = df_total.drop('diapo', axis = 1)

df_total = df_total.groupby(groupby_features).agg(['mean']).reset_index()
df_total.columns= df_total.columns.map('_'.join).str.strip('_')


In [73]:
y = pd.concat([df_total[['stress']], df_total[['video_name']]], axis = 1 )
x = df_total.drop(['stress_global',	'stress','frameTimeWindow'], axis = 1)
x.sexe = x.sexe.replace('H',0)
x.sexe = x.sexe.replace('F',1)

In [74]:
from sklearn.model_selection import LeaveOneGroupOut

groups = df_total['video_name']
loo = LeaveOneGroupOut()

cv_loo = loo.split(x, y, groups)


In [78]:
x_no_name = x.drop(['video_name'], axis = 1)
y_no_name = y.drop(['video_name'], axis = 1)


In [79]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

model = RandomForestClassifier(random_state = 42)
y_pred= cross_val_predict(model, x_no_name, y_no_name, cv=cv_loo)

In [80]:
print("F1 score = ", f1_score(y_no_name, y_pred, average='weighted'))

F1 score =  0.4247932417279817


In [ ]:
##fare per diapo et per timewindow e metterlo nel drive parametri. 